In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys
module_path = "../../0_lab_preparation"
sys.path.append(os.path.abspath(module_path))

from common import check_kernel
check_kernel()

## Convert multi-turn conversation to single-turn conversation

In [ ]:
import json

input_file = "../../1_synthetic-qa-generation/seed/dataset/azure-ai-services-openai-en-46-76-oai.jsonl"
output_file = "../../1_synthetic-qa-generation/seed/dataset/azure-ai-services-openai-en-46-76-oai-single.jsonl"

with open(input_file, "r", encoding='utf-8-sig') as fin, open(output_file, "w", encoding="utf-8") as fout:
    for line in fin:
        line = line.strip()
        if not line:
            continue
        data = json.loads(line)
        messages = data.get("messages", [])
        
        # systemメッセージが先頭に来ていることを前提
        # messages例：
        # [{"role": "system", "content": ...},
        #  {"role": "user", "content": ...},
        #  {"role": "assistant", "content": ...},
        #  {"role": "user", "content": ...},
        #  {"role": "assistant", "content": ...}]
        
        # 最初に出てくるuser→assistantペアのみを抽出する
        # systemは残したい場合は残す（ここでは残す想定）
        
        filtered_messages = []
        # systemメッセージをそのまま取り込む（必須でなければ省略可）
        if messages and messages[0]["role"] == "system":
            filtered_messages.append(messages[0])
            start_index = 1
        else:
            start_index = 0
        
        # 最初のuserメッセージと、その直後のassistantメッセージのみ抽出
        # userとassistantを1ペア見つけたらループを抜ける
        user_found = False
        for i in range(start_index, len(messages)):
            if messages[i]["role"] == "user" and not user_found:
                filtered_messages.append(messages[i])
                user_found = True
            elif messages[i]["role"] == "assistant" and user_found:
                filtered_messages.append(messages[i])
                # 1ペアのみ欲しいのでbreak
                break
        
        # filtered_messagesが3件（system,user,assistant）になるのが理想
        # user->assistantペアのみでもよければsystemを初めからスキップする
        
        if len(filtered_messages) >= 2:
            # 新たなJSONL形式で出力
            json.dump({"messages": filtered_messages}, fout, ensure_ascii=False)
            fout.write("\n")


## Encode checker

In [ ]:
def has_bom(file_path):
    """
    ファイルがBOMを含んでいるかをチェックする関数。

    Args:
        file_path (str): チェックするファイルのパス。

    Returns:
        bool: BOMを含んでいる場合はTrue、そうでない場合はFalse。
    """
    BOM = b'\xef\xbb\xbf'  # UTF-8 BOMのバイト列
    with open(file_path, 'rb') as file:
        first_bytes = file.read(3)
    return first_bytes == BOM

# 使用例
# file_path = input_file
file_path = "dataset/train.jsonl"
if has_bom(file_path):
    print(f"The file '{file_path}' contains a BOM.")
else:
    print(f"The file '{file_path}' does not contain a BOM.")


In [ ]:
import string

def contains_special_characters(file_path):
    """
    ファイルに特殊文字（制御文字や非印刷可能文字）が含まれているか確認する関数。

    Args:
        file_path (str): チェックするファイルのパス。

    Returns:
        list: 特殊文字が含まれる行番号とその内容をリストで返す。
    """
    # 許容される文字（印刷可能なASCII文字と一般的な制御文字: \n, \t, \r）
    allowed_chars = set(string.printable) - set(string.whitespace) | {'\n', '\t', '\r'}
    
    special_char_lines = []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line_number, line in enumerate(file, start=1):
            for char in line:
                if char not in allowed_chars:
                    special_char_lines.append((line_number, line.strip()))
                    break  # その行に1つでも特殊文字があれば次の行へ進む

    return special_char_lines


# 使用例
file_path = "dataset/train.jsonl"
special_chars = contains_special_characters(file_path)
if special_chars:
    print("Special characters detected in the following lines:")
    for line_number, content in special_chars:
        print(f"Line {line_number}: {content}")
else:
    print("No special characters found in the file.")


In [ ]:
import unicodedata

def check_encoding_issues(file_path):
    """
    LLMのFine-tuningに影響を与えるエンコード問題をチェックする関数。

    チェック内容:
    1. BOMの有無
    2. 不正または非標準のUnicode文字（制御文字など）

    Args:
        file_path (str): チェックするファイルのパス。

    Returns:
        dict: BOMの有無とエンコード問題が含まれる行情報を含む辞書。
    """
    BOM = b'\xef\xbb\xbf'
    results = {"has_bom": False, "encoding_issues": []}
    
    # BOMチェック
    with open(file_path, 'rb') as file:
        first_bytes = file.read(3)
    results["has_bom"] = (first_bytes == BOM)

    # 不正なUnicode文字のチェック
    with open(file_path, 'r', encoding='utf-8-sig') as file:
        for line_number, line in enumerate(file, start=1):
            for char in line:
                # Unicode制御文字（スペースや改行以外）を検出
                if unicodedata.category(char).startswith("C") and char not in {'\n', '\t', '\r'}:
                    results["encoding_issues"].append((line_number, line.strip()))
                    break  # 問題がある行を1つだけ記録して次の行に進む
    
    return results


# 使用例
file_path = "dataset/train.jsonl"
check_results = check_encoding_issues(file_path)

if check_results["has_bom"]:
    print("The file contains a BOM, which may affect fine-tuning.")

if check_results["encoding_issues"]:
    print("Encoding issues detected in the following lines:")
    for line_number, content in check_results["encoding_issues"]:
        print(f"Line {line_number}: {content}")
else:
    print("No encoding issues found in the file.")


In [ ]:
def remove_bom(file_path, output_path=None):
    """
    ファイルからBOMを排除する関数。

    Args:
        file_path (str): 元のファイルのパス。
        output_path (str, optional): BOMを削除したファイルの保存先。
                                     指定しない場合、元のファイルを上書き。

    Returns:
        bool: BOMが削除された場合はTrue、元々含まれていない場合はFalse。
    """
    BOM = b'\xef\xbb\xbf'  # UTF-8のBOM
    with open(file_path, 'rb') as file:
        content = file.read()
    
    # BOMが含まれているかチェック
    if content.startswith(BOM):
        content = content[len(BOM):]  # BOMを除去
        if output_path is None:  # 上書き保存
            output_path = file_path
        with open(output_path, 'wb') as file:
            file.write(content)
        print(f"BOM removed from '{file_path}' and saved to '{output_path}'.")
        return True
    else:
        print(f"No BOM found in '{file_path}'.")
        return False

# 使用例
file_path = "../../1_synthetic-qa-generation/seed/dataset/azure-ai-services-openai-pages-1-to-77-oai.jsonl"
output_path = "../../1_synthetic-qa-generation/seed/dataset/azure-ai-services-openai-pages-1-to-77-oai-no-bow.jsonl"
remove_bom(file_path, output_path)
